In [1]:
import numpy as np
import simpy 
from scipy.stats import expon

In [2]:
np.random.seed(42)

def arrival(env,server1,server2,S,G):
    while True:
        dt = expon.rvs(scale = 0.5)
        arrival = env.timeout(dt)
        yield arrival
        S.append(env.now)
        env.process(service(env,server1,server2,S,G))
    
def service(env, server1,server2, S,G):
    rqt1 = server1.request()
    rqt2 = server2.request()
    result = yield rqt1 | rqt2
    if rqt1 in result:
        print("customer goes to server 1")
        rqt2.cancel()
        dt = expon.rvs(scale = 1/2)
        yield env.timeout(dt)
        server1.release(rqt1)
    else:
        print("customer goes to server 2")
        rqt1.cancel()
        dt= expon.rvs(scale = 1)
        yield env.timeout(dt)
        server1.release(rqt2)
    G.append(env.now)
    
    
def simulate():
    S = []
    G = []
    env = simpy.Environment()
    server1 = simpy.Resource(env,capacity=1)
    server2 = simpy.Resource(env,capacity = 1)
    env.process(arrival(env,server1,server2,S,G))
    env.run(until=3)
    return S,G


simulate()

customer goes to server 1
customer goes to server 1
customer goes to server 1
customer goes to server 1


([0.23463404498842955,
  1.7396947604471902,
  2.196166037335167,
  2.280964183292469],
 [0.8930068917611542,
  1.8245071956783634,
  2.226085421639507,
  2.7405052601061013])